<a href="https://colab.research.google.com/github/bspanswick/STLF/blob/main/00_STLF_DataPrep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm

import math 
from datetime import date, timedelta, datetime 
from pandas.plotting import register_matplotlib_converters 
import matplotlib.dates as mdates 
from sklearn.metrics import mean_absolute_error, mean_squared_error 
from keras.models import Sequential 
from keras.layers import LSTM, Dense, Dropout 
from keras.callbacks import EarlyStopping 
from sklearn.preprocessing import RobustScaler, MinMaxScaler 
from fbprophet import Prophet
from sklearn.metrics import mean_absolute_error
from matplotlib import pyplot
from pandas import DataFrame
from statsmodels.tsa.api import VAR

import os
os.chdir('/content/drive/MyDrive/AMTLF/')

In [20]:
weather = pd.read_csv('weather.csv')
pjm = pd.read_csv('pjm.csv')

In [6]:
weather.head()

,DATA_DATE_and_HOUR,TEMPERATURE,WIND_SPEED_MPH,WIND_GUST_MPH,RELATIVE_HUMIDITY,CLOUD_COVER,PRECIPITATION
0,01/Jan/2015 01,17.0,12.0,12.0,49.0,19.0,0.0
1,01/Jan/2015 02,17.0,12.0,12.0,49.0,10.0,0.0
2,01/Jan/2015 03,17.0,14.0,14.0,51.0,0.0,0.0
3,01/Jan/2015 04,16.0,12.0,12.0,54.0,0.0,0.0
4,01/Jan/2015 05,15.0,7.0,7.0,56.0,0.0,0.0


In [7]:
pjm.head()

,datetime_beginning_utc,datetime_beginning_ept,nerc_region,mkt_region,zone,load_area,mw,is_verified
0,2017-01-01T20:00:00.000Z,2017-01-01T15:00:00.000Z,RFC,MIDATL,PL,PLCO,4039.059,True
1,2017-01-01T22:00:00.000Z,2017-01-01T17:00:00.000Z,RFC,MIDATL,PL,PLCO,4658.056,True
2,2017-01-02T08:00:00.000Z,2017-01-02T03:00:00.000Z,RFC,MIDATL,PL,PLCO,3854.594,True
3,2017-01-02T11:00:00.000Z,2017-01-02T06:00:00.000Z,RFC,MIDATL,PL,PLCO,4269.468,True
4,2017-01-02T13:00:00.000Z,2017-01-02T08:00:00.000Z,RFC,MIDATL,PL,PLCO,4746.482,True


In [8]:
#set datetimes and get min/max/mean for days

weather = weather.rename(columns={ weather.columns[0]: "datetime" })
weather.datetime = pd.to_datetime(weather['datetime'])
weather['time'],weather['date']= weather['datetime'].apply(lambda x:x.time()), weather['datetime'].apply(lambda x:x.date())
weather1 = weather.groupby('date').agg({'TEMPERATURE': ['mean', 'min', 'max'],
                                       'RELATIVE_HUMIDITY': ['mean'],
                                       'WIND_GUST_MPH': ['mean'],
                                       'CLOUD_COVER': ['mean'],
                                       'PRECIPITATION': ['mean']})
#convert back to usable dataframe
weather1.columns = ['_'.join(col) for col in weather1.columns]
weather1 = weather1.reset_index()
weather1.date = pd.to_datetime(weather1['date'])

In [9]:
#get day of week and check for federal holiday

weather1['weekday'] =weather1['date'].dt.dayofweek
weather1['month'] = weather1['date'].dt.month
cal = calendar()
holidays = cal.holidays(start=weather1.date.min(), end=weather1.date.max())
weather1['holiday'] = weather1['date'].isin(holidays).astype(int)

In [10]:
pjm = pjm.rename(columns={ pjm.columns[1]: "datetime" })
pjm.datetime = pd.to_datetime(pjm['datetime'])
pjm['time'],pjm['date']= pjm['datetime'].apply(lambda x:x.time()), pjm['datetime'].apply(lambda x:x.date())
pjm1 = pjm.groupby('date').agg({'mw': ['max']})
pjm1.columns = ['_'.join(col) for col in pjm1.columns]
pjm1 = pjm1.reset_index()
pjm1.date = pd.to_datetime(pjm1['date'])

In [11]:
weather1.head()

,date,TEMPERATURE_mean,TEMPERATURE_min,TEMPERATURE_max,RELATIVE_HUMIDITY_mean,WIND_GUST_MPH_mean,CLOUD_COVER_mean,PRECIPITATION_mean,weekday,month,holiday
0,2015-01-01,28.878261,14.0,42.0,54.343478,11.800000,5.686957,0.0000,3,1,1
1,2015-01-02,31.508333,19.0,43.0,61.887500,6.787500,32.333333,0.0000,4,1,0
2,2015-01-03,29.845833,18.0,40.0,80.008333,5.262500,77.025000,47.8500,5,1,0
3,2015-01-04,40.725000,34.0,58.0,78.987500,9.333333,94.629167,30.1125,6,1,0
4,2015-01-05,29.687500,10.0,45.0,56.333333,27.516667,46.662500,0.4125,0,1,0


In [12]:
#checking some of the data for missing dates, and date ranges

print(pjm1.date.max())
print(weather1.date.min())
print(pd.date_range(start = weather1.date.min(), end = weather1.date.max() ).difference(weather1.date))
print(pd.date_range(start = weather1.date.min(), end = weather1.date.max() ).difference(pjm1.date))


2019-12-31 00:00:00
2015-01-01 00:00:00
DatetimeIndex([], dtype='datetime64[ns]', freq=None)
DatetimeIndex([], dtype='datetime64[ns]', freq=None)


In [13]:
df = pd.merge(weather1, pjm1, on='date', how='outer')
df = df.dropna()
#df['date'] = pd.to_datetime(df['date'])
#df = df.set_index('date')
#df = df.reset_index()
df.head()

,date,TEMPERATURE_mean,TEMPERATURE_min,TEMPERATURE_max,RELATIVE_HUMIDITY_mean,WIND_GUST_MPH_mean,CLOUD_COVER_mean,PRECIPITATION_mean,weekday,month,holiday,mw_max
0,2015-01-01,28.878261,14.0,42.0,54.343478,11.800000,5.686957,0.0000,3.0,1.0,1.0,5214.883
1,2015-01-02,31.508333,19.0,43.0,61.887500,6.787500,32.333333,0.0000,4.0,1.0,0.0,5464.446
2,2015-01-03,29.845833,18.0,40.0,80.008333,5.262500,77.025000,47.8500,5.0,1.0,0.0,5572.899
3,2015-01-04,40.725000,34.0,58.0,78.987500,9.333333,94.629167,30.1125,6.0,1.0,0.0,4985.939
4,2015-01-05,29.687500,10.0,45.0,56.333333,27.516667,46.662500,0.4125,0.0,1.0,0.0,6336.146


In [21]:
df.to_csv('cleandf.csv', encoding='utf-8', index=False)